<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**

# Indexation d'images

Vous êtes bénévole pour l'association de protection des animaux de votre quartier. C'est d'ailleurs ainsi que vous avez trouvé votre compagnon idéal, Snooky. Du coup, vous vous demandez ce que vous pouvez faire en retour pour aider l'association.

Vous apprenez, en discutant avec un bénévole, que leur base de données de pensionnaires commence à s'agrandir et qu'ils n'ont pas toujours le temps de référencer les images des animaux qu'ils ont accumulées depuis plusieurs années. Ils aimeraient donc réaliser un index de l’ensemble de la base de données d’images qu’ils possèdent, pour classer les chiens par races.

**Les données**

Les bénévoles de l'association n'ont pas eu le temps de réunir les différentes images des pensionnaires dispersées sur leurs disques durs. Pas de problème, vous développerez un algorithme en utilisant le Stanford Dogs Dataset pour entraîner votre algorithme.

**Votre mission**

En tant que Data Scientist, l'association vous demande de réaliser un algorithme de détection de la race du chien sur une photo, afin d'accélérer leur travail d’indexation.

**Contraintes**

Lors de ce projet, vous mettrez en œuvre deux approches.

* Une approche classique : il s’agit de pre-processer des images avec des techniques spécifiques (e.g.whitening, equalisation, filtre linéaire/laplacien/gaussien, éventuellement modifier la taille des images), puis d’extraire des features (e.g. texture, corners, edges et SIFT detector). Il faut ensuite réduire les dimensions, soit par des approches classiques (e.g. PCA, k-means) soit avec une approche par histogrammes et dictionary learning (bag-of-words appliqué aux images), puis appliquer des algorithmes de classification standards.



* Lors de l’analyse exploratoire, vous regarderez si les features extraites et utilisées en classification sont prometteuses en utilisant des méthodes de réduction de dimension pour visualiser le dataset en 2D. Cela vous permettra d’affiner votre intuition sur les différents traitements possibles, sans que cela ne se substitue à des mesures de performances rigoureuses.



* Une approche s’appuyant sur l’état de l’art et l’utilisation de CNN (réseaux de neurones convolutionnels). Compte tenu de la taille et de la complexité du dataset, et de la puissance de calcul à votre disposition, il est très difficile d’obtenir de bonnes performances (pour ça, essayez MNIST). Aussi, est-il recommandé d’utiliser le transfer learning, c’est-à-dire utiliser un réseau déjà entraîné, et le modifier pour répondre à votre problème. Une première chose obligatoire est de ré-entraîner les dernières couches pour prédire les classes qui vous intéressent seulement. Il est également possible d’adapter la structure (supprimer certaines couches par exemple) ou de ré-entraîner le modèle avec un très faible learning rate pour ajuster les poids à votre problème (plus long) et optimiser les performances.

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [43]:
import os
import random

#-------------------------------------------------------------------------------
# List of all directories, each directory conatains matches with a breed dog
#-------------------------------------------------------------------------------
list_dir = os.listdir('./data/Images')

#-------------------------------------------------------------------------------
# Dogs breeds are extracted from file name and loaded into a dictionary 
# structured as fllowing : {dir_name:list_of_images}
#-------------------------------------------------------------------------------
dict_breed_all = { breed_dir_name :breed_dir_name.split('-')[1] for breed_dir_name in list_dir }
list_breed_all = [breed_dir_name.split('-')[1] for breed_dir_name in list_dir]

print("Number of dog breeds= {}".format(len(dict_breed_all)))

#-------------------------------------------------------------------------------
# A random list of directories are selected among dogs breeds
# list_breed_sample contains the sampled list of dogs breeds.
#-------------------------------------------------------------------------------
breed_count=13
dog_breed_count=4

list_breed_sample = list()

for sample in range(0, breed_count,1):
    list_breed_sample.append(random.randrange(0, len(dict_breed_all), int(len(dict_breed_all)/breed_count)))
    
list_breed_sample,len(list_breed_sample)

#-------------------------------------------------------------------------------
# List of name containing directories is built and duplicated names are removed
#-------------------------------------------------------------------------------
list_breed_sample_name =[list_breed_all[index] for index in list_breed_sample]
list_breed_sample_name = list(set(list_breed_sample_name))
list_breed_sample_name

Number of dog breeds= 120


['affenpinscher',
 'miniature_schnauzer',
 'Sealyham_terrier',
 'German_shepherd',
 'otterhound',
 'briard',
 'Saint_Bernard',
 'toy_poodle',
 'miniature_poodle']

In [56]:
import p7_util
from PIL import Image

#-------------------------------------------------------------------------------
# From any of the dogs breeds, a sample of dogs is selected.
# Means, images from these sampling are read from files.
#-------------------------------------------------------------------------------

dict_breed_image = dict()
dict_breed_filename = dict()


for directory, breed in dict_breed_all.items() :
    if directory.split('-')[1] in list_breed_sample_name :
        print(directory)
        list_image = list()
        list_filename = list()
        list_image_file = os.listdir('./data/Images'+'/'+directory)
        for index_image_file in range(0,dog_breed_count,1) :
            filename = './data/Images/'+directory+'/'+str(list_image_file[index_image_file])
            list_filename.append(filename)
            list_image.append(p7_util.p7_image_load(filename, is_verbose=False))
        dict_breed_image[breed] = list_image
        dict_breed_filename[breed] = list_filename
len(dict_breed_image)

n02113624-toy_poodle
n02106662-German_shepherd
n02091635-otterhound
n02109525-Saint_Bernard
n02105251-briard
n02110627-affenpinscher
n02095889-Sealyham_terrier
n02097047-miniature_schnauzer
n02113712-miniature_poodle


9

In [57]:
from IPython.display import Image, HTML, display

for breed, list_filename in dict_breed_filename.items() :
    print(breed)
    imagesList=''.join( ["<img style='width: 100px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                     for s in list_filename ])
    display(HTML(imagesList))


toy_poodle


German_shepherd


otterhound


Saint_Bernard


briard


affenpinscher


Sealyham_terrier


miniature_schnauzer


miniature_poodle
